In [1]:
import torch

print(torch.__version__)

device = torch.device("cuda:0" if torch.cuda.is_available else "cpu")
print(device)
#device=torch.device('cpu')########

1.3.1
cuda:0


In [3]:
import os

corpus_name = "train_model32000.txt"#"mecab_tagged.txt"#
position = "./data"
corpus = os.path.join(position, corpus_name)
print(corpus)

./data/train_model32000.txt


In [5]:
from Voc import *

In [20]:
printLines(corpus)

['▁なんと !', '▁なんと !\n'] 2 2
['▁なんと !', '▁ 意味 : 都 を 移 すこ と 、 都 移 り\n'] 2 12
['▁わたし 安定の たっくん ですよー 気づいたら たっくん いないし 、 いつき あるし 誰 と交換 したのか', '▁まさかの 増えてる w 羨ましい\n'] 13 4
['▁まさかの 増えてる w 羨ましい', '▁ 連れ が 誰か 引いて た んですよ 、 きっと それ と交換 したんだ と思うんですけど 誰 だったのか w あ 、 たっくん は しれっと 買取 に 出しました ー\n'] 4 25
['▁ おち つけ よ', '▁みんな は げろ\n'] 4 3
['▁みんな は げろ', '▁昨日 髪の毛 切った ばっかり だけど このくらい つる っ パ ゲ になって きます !\n'] 3 13
['▁今は 古 すぎて 廃 盤 のもの が多いので 、 いい モンスター が出る の だと 高い 値 で 売れ ます w', '▁あ 、 やっぱり そう言う のある ですね w\n'] 19 7
['▁あ 、 やっぱり そう言う のある ですね w', '▁ あります あります w\n'] 7 4
['▁やだ お', '▁ 目元 は 知ってる 、 かあいい\n'] 2 6
['▁ 目元 は 知ってる 、 かあいい', '▁超 過去 写真\n'] 6 3


In [21]:
MIN_COUNT = 28#22#3#
print(corpus_name)
save_dir = os.path.join("data", "save")
voc, pairs = loadPrepareData(corpus, corpus_name)#, datafile, save_dir)
pairs = trimRareWords(voc, pairs, MIN_COUNT)
# Print some pairs to validate
print("\npairs:")
for pair in pairs[:10]:
    print(pair)

train_model32000.txt
Start preparing training data ...
Reading lines...
Read 2187212 sentence pairs
Trimmed to 1834337 sentence pairs
Counting words...
Counted words: 34296
keep_words 31995 / 34293 = 0.9330
Trimmed from 1834337 pairs to 1822478, 0.9935 of total

pairs:
['▁なんと !', '▁なんと !']
['▁なんと !', '▁ 意味 : 都 を 移 すこ と 、 都 移 り']
['▁わたし 安定の たっくん ですよー 気づいたら たっくん いないし 、 いつき あるし 誰 と交換 したのか', '▁まさかの 増えてる w 羨ましい']
['▁ おち つけ よ', '▁みんな は げろ']
['▁みんな は げろ', '▁昨日 髪の毛 切った ばっかり だけど このくらい つる っ パ ゲ になって きます !']
['▁今は 古 すぎて 廃 盤 のもの が多いので 、 いい モンスター が出る の だと 高い 値 で 売れ ます w', '▁あ 、 やっぱり そう言う のある ですね w']
['▁あ 、 やっぱり そう言う のある ですね w', '▁ あります あります w']
['▁やだ お', '▁ 目元 は 知ってる 、 かあいい']
['▁ 目元 は 知ってる 、 かあいい', '▁超 過去 写真']
['▁ は ?', '▁クソ じゃん 気にし ねー で いいんじゃね w まあ 俺は ファイヤー 党 の人 なので もちろん あの方 に入れた が な']


In [22]:
import pickle
# #Load batches for each iteration
# train_pairs,test_pairs = train_test_split(pairs ,test_size=0.2)
# with open('data/train_test_pairs_mecab.pickle', 'wb') as f:
#     pickle.dump((train_pairs,test_pairs), f)
# with open("data/train_test_pairs_mecab3min.pickle", "rb") as f:
#     train_pairs,test_pairs = pickle.load(f)
with open("data/train_test_pairs_32k.pickle", "rb") as f:
    train_pairs,test_pairs = pickle.load(f)
 
print(voc.name)
print("ちゃんと目的のコーパスファイルを読み込んでいるか確認")
print("train size",len(train_pairs))
print("example",train_pairs[:2])
print("test size",len(test_pairs))
print("example",test_pairs[:2])

train_model32000.txt
ちゃんと目的のコーパスファイルを読み込んでいるか確認
train size 1457982
example [['▁ 問い合わせ w 間違いない ですね w', '▁ して はいない んですけどね w それも 運 なの だろう'], ['▁ 珈琲 とか ウイスキー 用', '▁ 娘 用']]
test size 364496
example [['▁ ピカチュウ は かわいい でも カビゴン はもっと かわいい', '▁いやあ ずみ のほうが'], ['▁ スカ トロ とか やめてくれ だめ や', '▁そこまで 言って ねーよ w それは あかん']]


In [10]:
# from model import *
from model_lstm import *

# model.pyのモデルの説明

Get embedding of current input word.
Forward through unidirectional GRU.
Calculate attention weights from the current GRU output from (2).
Multiply attention weights to encoder outputs to get new “weighted sum” context vector.
Concatenate weighted context vector and GRU output using Luong eq. 5.
Predict next word using Luong eq. 6 (without softmax).
Return output and final hidden state.

input_step: one time step (one word) of input sequence batch; shape=(1, batch_size)(1,3)
last_hidden: final hidden layer of GRU; shape=(n_layers x num_directions, batch_size, hidden_size)(1,3,2)
encoder_outputs: encoder model’s output; shape=(max_length, batch_size, hidden_size)(4,3,2)

In [11]:
#訓練するときだけ
print(corpus_name)
# Configure models
model_name = "focal_loss_model"
attn_model = 'dot'
# attn_model = 'general'
# attn_model = "concat"
hidden_size = 256
encoder_n_layers = 4
decoder_n_layers = 4
dropout = 0.1
batch_size = 512
teacher_forcing_ratio=0.2

# Set checkpoint to load from; set to None if starting from scratch
loadFilename = None
checkpoint_iter = 35000
loadFilename = os.path.join(save_dir, model_name, corpus_name,
                           '{}-{}_{}_{}'.format(encoder_n_layers, decoder_n_layers, hidden_size, teacher_forcing_ratio),
                           '{}_checkpoint.tar'.format(checkpoint_iter))


# Load model if a loadFilename is provided
if loadFilename:
    print("load save file")
    # If loading on same machine the model was trained on
    checkpoint = torch.load(loadFilename)
    # If loading a model trained on GPU to CPU
    # checkpoint = torch.load(loadFilename, map_location=torch.device('cpu'))
    encoder_sd = checkpoint["en"]
    decoder_sd = checkpoint["de"]
    encoder_optimizer_sd = checkpoint["en_opt"]
    decoder_optimizer_sd = checkpoint["de_opt"]
    embedding_sd = checkpoint["embedding"]
    voc.__dict__ = checkpoint["voc_dict"]


print("Building encoder and decoder ...")
# Initialize word embeddings
embedding = nn.Embedding(voc.num_words, hidden_size)
if loadFilename:
    embedding.load_state_dict(embedding_sd)
# Initialize encoder & decoder models
encoder = EncoderRNN(hidden_size, embedding, encoder_n_layers, dropout)
decoder = AttnDecoderRNN(
    attn_model, embedding, hidden_size, voc.num_words, decoder_n_layers, dropout
)
if loadFilename:
    encoder.load_state_dict(encoder_sd)
    decoder.load_state_dict(decoder_sd)
# Use appropriate device
encoder = encoder.to(device)
decoder = decoder.to(device)
print("Models built and ready to go!")

train_model32000.txt
load save file
Building encoder and decoder ...
Models built and ready to go!


In [12]:
# パラメータカウント関数
print("パラメータ数をカウント")
print("encoderのパラメータ数")
parameters_count(encoder)
print("decoderパラメータ数")
parameters_count(decoder)
#print(encoder)

パラメータ数をカウント
encoderのパラメータ数
13975040
decoderパラメータ数
18651646


In [13]:
class GreedySearchDecoder(nn.Module):
    def __init__(self, encoder, decoder):
        super(GreedySearchDecoder, self).__init__()
        self.encoder = encoder
        self.decoder = decoder

    def forward(self, input_seq, input_length, max_length):
        # Forward input through encoder model
        encoder_outputs, encoder_hidden = self.encoder(input_seq, input_length)
        # Prepare encoder's final hidden layer to be first hidden input to the decoder
        decoder_hidden = encoder_hidden[: decoder.n_layers]
        # Initialize decoder input with SOS_token
        decoder_input = torch.ones(1, 1, device=device, dtype=torch.long) * SOS_token
        # Initialize tensors to append decoded words to
        all_tokens = torch.zeros([0], device=device, dtype=torch.long)
        all_scores = torch.zeros([0], device=device)
        # Iteratively decode one word token at a time
        for _ in range(max_length):
            # Forward pass through decoder
            decoder_output, decoder_hidden = self.decoder(
                decoder_input, decoder_hidden, encoder_outputs
            )
            # Obtain most likely word token and its softmax score
            decoder_scores, decoder_input = torch.max(decoder_output, dim=1)
            # Record token and score
            all_tokens = torch.cat((all_tokens, decoder_input), dim=0)
            all_scores = torch.cat((all_scores, decoder_scores), dim=0)
            # Prepare current token to be next decoder input (add a dimension)
            decoder_input = torch.unsqueeze(decoder_input, 0)
        # Return collections of word tokens and scores
        return [all_tokens], [-torch.sum(torch.log(all_scores).detach().cpu())/float(max_length - 1 + 1e-6)]
    
    
import sentencepiece as spm
segmentation_model_position = './data'
segmentation_model_name = 'train_model32000.model'

spp = spm.SentencePieceProcessor()
spp.Load(os.path.join(segmentation_model_position, segmentation_model_name))
def normalizeString(input_sentence):
    splitSentence = ' '.join(spp.EncodeAsPieces(input_sentence))
    return splitSentence

# import MeCab
# tagger = MeCab.Tagger("-d /usr/lib/x86_64-linux-gnu/mecab/dic/mecab-ipadic-neologd/")

# def normalizeString(input_sentence):
#     splitSentence = ' '.join([ele.split("\t")[0] for ele in tagger.parse(input_sentence).split("\n")[:-2]])
#     return splitSentence

In [14]:

class BeamSearchDecoder(nn.Module):
    def __init__(self, encoder, decoder, beam_width, n_best):
        super(BeamSearchDecoder, self).__init__()
        
        self.encoder = encoder
        self.decoder = decoder
        self.beam_width = beam_width
        self.n_best = n_best
        
    def forward(self, input_seq, input_length, max_length):
        # Forward input through encoder model
        encoder_outputs, encoder_hidden = self.encoder(input_seq, input_length)
        # Prepare encoder's final hidden layer to be first hidden input to the decoder
        decoder_hidden = encoder_hidden[: decoder.n_layers]
        # Initialize decoder input with SOS_token
        #decoder_input = torch.ones(1, 1, device=device, dtype=torch.long) * SOS_token
        
        
        class BeamSearchNode(object):
            def __init__(self, hid, prevNode, wordId, logP, length):
                self.hid = hid
                self.prevNode = prevNode
                self.wordId = wordId
                self.logP = logP
                self.length = length
                
            def eval(self, alpha=1.0):
                reward = 0
                # Add here a function for shaping a reward

                return self.logP / float(self.length - 1 + 1e-6) + alpha * reward
          
        
        n_best_batch_list = []
        n_best_batch_score_list = []
        for batchNum in range(encoder_hidden.size(1)):
            
            decoder_input = torch.LongTensor([[SOS_token]]).to(device)
            # 一つ前の状態の隠れベクトル、単語をNodeを保持するNodeを生成
            node = BeamSearchNode(hid=decoder_hidden[:,batchNum,:].unsqueeze(1), prevNode=None, wordId=decoder_input, logP=0, length=1)
            nextNodes=[]
            
            nextNodes.append((-node.eval(), id(node), node))
            n_dec_steps = 0
            while True:
                
                nodes = [sorted(nextNodes)[inode] for inode in range(len(nextNodes)) if inode<self.beam_width]
                nextNodes = []
                end_node = []
                for beamNum in range(self.beam_width):
                    if len(nodes)<=0:
                        break
                    #今から探索するNodeを取得
                    score, _, n = nodes.pop(0)
                    decoder_input = n.wordId
                    decoder_hidden = n.hid
                    
                    if n.wordId[0][0].item()!=EOS_token and n.wordId[0][0].item()!=PAD_token and n.length<=max_length:
                        decoder_output, decoder_hidden = self.decoder(
                            decoder_input, decoder_hidden, encoder_outputs
                        )

                        topk_prob, topk_indexes = torch.topk(decoder_output, self.beam_width) 

                        for new_k in range(self.beam_width):
                            decoded_t = topk_indexes[0][new_k].view(1,1) # (1)
                            logp = torch.log(topk_prob[0][new_k]).item() # float log probability val

                            node = BeamSearchNode(hid=decoder_hidden,
                                                  prevNode=n,
                                                  wordId=decoded_t,
                                                  logP=n.logP+logp,
                                                  length=n.length+1)
                            nextNodes.append((-node.eval(), id(node), node))
                            
                            
                    else:
                        node = BeamSearchNode(hid=decoder_hidden,
                                                  prevNode=n,
                                                  wordId=torch.LongTensor([[PAD_token]]).to(device),
                                                  logP=n.logP,
                                                  length=n.length)
                        nextNodes.append((-node.eval(), id(node), node))
                        end_node.append(1)
                if len(end_node)>=self.beam_width:
                    break
            
            
        
            if len(nextNodes)!=self.beam_width:
                print('assert not match beam width and nextNode length')
                
            n_best_seq_list = []
            n_best_score_list = []
            for score, _id, n in sorted(nextNodes):
                sequence = [n.wordId.item()]
                n_best_score_list.append(score)
                # back trace from end node
                while n.prevNode is not None:
                    n = n.prevNode
                    sequence.append(n.wordId.item())
                    
                sequence = sequence[::-1] # reverse
                n_best_seq_list.append(sequence)
        
            n_best_seq_list = n_best_seq_list[::-1]
            n_best_score_list = n_best_score_list[::-1]
            
            n_best_batch_list.append(n_best_seq_list)
            n_best_batch_score_list.append(n_best_score_list)
            
        n_best_batch_list = torch.tensor(n_best_batch_list)
        n_best_batch_score_list = torch.tensor(n_best_batch_score_list)
            
            #batchを無視して今回は出力することにした
        return n_best_batch_list[0], n_best_batch_score_list[0]
    
# import sentencepiece as spm
# segmentation_model_position = './data'
# segmentation_model_name = 'train_model20000.model'

# spp = spm.SentencePieceProcessor()
# spp.Load(os.path.join(segmentation_model_position, segmentation_model_name))
# def normalizeString(input_sentence):
#     splitSentence = ' '.join(spp.EncodeAsPieces(input_sentence))
#     return splitSentence

In [15]:
def evaluate(encoder, decoder, searcher, voc, sentence, max_length=MAX_LENGTH):
    ### Format input sentence as a batch
    # words -> indexes
    indexes_batch = [indexesFromSentence(voc, sentence)]
    # Create lengths tensor
    lengths = torch.tensor([len(indexes) for indexes in indexes_batch])
    # Transpose dimensions of batch to match models' expectations
    input_batch = torch.LongTensor(indexes_batch).transpose(0, 1)
    # Use appropriate device
    input_batch = input_batch.to(device)
    lengths = lengths.to(device)
    # Decode sentence with searcher
    seqs, scores = searcher(input_batch, lengths, max_length)
    
    for iseqs in range(len(seqs)):
        tokens = seqs[iseqs]
    
        # indexes -> words
        decoded_words = [voc.index2word[token.item()] for token in tokens]
        print(scores[iseqs],decoded_words)
    return decoded_words


def evaluateInput(encoder, decoder, searcher, searcher1, voc):
    input_sentence = ""
    while 1:
        try:
            # Get input sentence
            input_sentence = input("> ")
            # Check if it is quit case
            if input_sentence == "q" or input_sentence == "quit":
                break
            # Normalize sentence
            input_sentence = normalizeString(input_sentence)
            print('\n入力を分割すると',input_sentence+'\n')
            
            # Evaluate sentence with seacher
            output_words = evaluate(encoder, decoder, searcher, voc, input_sentence)
            # Format and print response sentence
            output_words[:] = [
                x for x in output_words if not (x == "EOS" or x == "PAD" or x == "SOS")
            ]
            print("\nBot(BeamSearch):", " ".join(output_words))
            
            print()
            
            # Evaluate sentence with seacher1
            output_words = evaluate(encoder, decoder, searcher1, voc, input_sentence)
            # Format and print response sentence
            output_words[:] = [
                x for x in output_words if not (x == "EOS" or x == "PAD" or x == "SOS")
            ]
            print("\nBot(GreedySearch):", " ".join(output_words)+"\n")

        except KeyError:
            print("Error: Encountered unknown word.")

# 自分の入力で評価(会話)

In [16]:
encoder.eval()

decoder.eval()

# Initialize search module
#searcher = GreedySearchDecoder(encoder, decoder)
searcher = BeamSearchDecoder(encoder, decoder, 5, 5)
searcher１ = GreedySearchDecoder(encoder, decoder)

# Begin chatting (uncomment and run the following line to begin)
evaluateInput(encoder, decoder, searcher, searcher1, voc)

>  おはよう



入力を分割すると ▁おはよう



AttributeError: 'tuple' object has no attribute 'size'

# 複数文をランダムで入力して出力して評価

In [15]:
import random
num=5
test_choice = [(random.choice(test_pairs)[0].replace(' ',''),random.choice(test_pairs)[1].replace(' ','')) for i in range(num)]

encoder.eval()
decoder.eval()
# Initialize search module
#searcher = GreedySearchDecoder(encoder, decoder)
searcher = BeamSearchDecoder(encoder, decoder, 5, 5)

for input_pair in test_choice:
    
    input_sentence, output_sentence = input_pair
    print("入力:",input_sentence)
    input_sentence = normalizeString(input_sentence)
    print('入力を分割すると:',input_sentence)
    

    # Evaluate sentence with seacher
    output_words = evaluate(encoder, decoder, searcher, voc, input_sentence)
    # Format and print response sentence
    output_words[:] = [
        x for x in output_words if not (x == "EOS" or x == "PAD" or x=="SOS")
    ]
    print("\nBot(BeamSearch):", "".join(output_words))
    print("正解:",output_sentence+"\n\n")

入力: 通学路
入力を分割すると: 通学 路
tensor(3.1944) ['SOS', 'いい', 'EOS', 'PAD', 'PAD']
tensor(3.0795) ['SOS', 'そう', 'EOS', 'PAD', 'PAD']
tensor(2.6557) ['SOS', 'これ', 'だ', 'EOS', 'PAD']
tensor(2.5137) ['SOS', 'これ', 'は', 'EOS', 'PAD']
tensor(2.4725) ['SOS', 'これ', 'EOS', 'PAD', 'PAD']

Bot(BeamSearch): これ
正解: 絶品の大盛りと欲張り盛りです!


入力: 間違えたならないことを願うねw
入力を分割すると: 間違え た なら ない こと を 願う ね w
tensor(2.1689) ['SOS', 'すまそ', 'w', 'w', 'w', 'w', 'EOS', 'PAD']
tensor(2.0860) ['SOS', 'すまそ', 'w', 'w', 'w', 'EOS', 'PAD', 'PAD']
tensor(2.0609) ['SOS', 'うん', 'w', 'EOS', 'PAD', 'PAD', 'PAD', 'PAD']
tensor(1.9430) ['SOS', 'すまそ', 'w', 'w', 'EOS', 'PAD', 'PAD', 'PAD']
tensor(1.5871) ['SOS', 'すまそ', 'w', 'EOS', 'PAD', 'PAD', 'PAD', 'PAD']

Bot(BeamSearch): すまそw
正解: やれやれやれやれ


入力: 模型w確かにいいですねw
入力を分割すると: 模型 w 確か に いい です ね w
tensor(1.6738) ['SOS', 'こっち', 'の', 'こと', 'に', 'なっ', 'てる', 'の', 'か', 'と', '思っ', 'て', 'ない', 'w', 'EOS', 'PAD', 'PAD']
tensor(1.6613) ['SOS', 'こっち', 'の', 'こと', 'に', 'なっ', 'てる', 'の', 'か', 'と', '思い', 'まし', 'た', 'w', '

# 複数文を自動評価指標で評価、ここから下

In [13]:
def evaluate(encoder, decoder, searcher, voc, sentence, max_length=MAX_LENGTH):
    ### Format input sentence as a batch
    # words -> indexes
    indexes_batch = [indexesFromSentence(voc, sentence)]
    # Create lengths tensor
    lengths = torch.tensor([len(indexes) for indexes in indexes_batch])
    # Transpose dimensions of batch to match models' expectations
    input_batch = torch.LongTensor(indexes_batch).transpose(0, 1)
    # Use appropriate device
    input_batch = input_batch.to(device)
    lengths = lengths.to(device)
    # Decode sentence with searcher
    seqs, scores = searcher(input_batch, lengths, max_length)
    
    for iseqs in range(len(seqs)):
        tokens = seqs[iseqs]
    
        # indexes -> words
        decoded_words = [voc.index2word[token.item()] for token in tokens]
        #print(scores[iseqs],decoded_words)
    return decoded_words

In [14]:
# test_extract = [random.choice(test_pairs) for i in range(5000)]


# input_choices = [ele[0] for ele in test_extract]
# output_choices = [ele[1].replace(' ','') for ele in test_extract]


# eval_folder = 'mecab3min'
# with open(eval_folder+'/ref.txt','w') as f:
#     f.write('\n'.join(output_choices))
        

# encoder.eval()
# decoder.eval()
# searcher = BeamSearchDecoder(encoder, decoder, 5, 5)
# hyp_sentences = []
# i=0
# try:
    

#     for input_sentence in input_choices:
#         input_sentence = normalizeString(input_sentence)
#         output_words = evaluate(encoder, decoder, searcher, voc, input_sentence)
#         output_words[:] = [
#             x for x in output_words if not (x == "EOS" or x == "PAD" or x=="SOS")
#         ]
#         hyp_sentences.append("".join(output_words))

#         i+=1
#     with open(eval_folder+'/hyp.txt','w') as f:
#         f.write('\n'.join(hyp_sentences))
# except Exception:
#     print('error')
#     print(i,hyp_sentences[-1])
#     print(input_sentence)
    
    

In [15]:
eval_folder = 'examples'
from nlgeval import compute_metrics
metrics_dict = compute_metrics(hypothesis=eval_folder+'/hyp.txt',
                               references=[eval_folder+'/ref1.txt'])


Bleu_1: 0.550000
Bleu_2: 0.428174
Bleu_3: 0.284043
Bleu_4: 0.201143
METEOR: 0.295797
ROUGE_L: 0.522104
CIDEr: 2.001651
SkipThoughtsCosineSimilarity: 0.626149
EmbeddingAverageCosineSimilarity: 0.866522
EmbeddingAverageCosineSimilairty: 0.866522
VectorExtremaCosineSimilarity: 0.558104
GreedyMatchingScore: 0.779369


In [16]:
import pickle
eval_folder = 'mecab3min'
# test_extract = [random.choice(test_pairs) for i in range(5000)]


# input_choices = [ele[0] for ele in test_extract]
# output_choices = [ele[1].replace(' ','') for ele in test_extract]


# with open(eval_folder+'/eval_input_output.pickle', 'wb') as f:
#     pickle.dump((input_choices ,output_choices), f)

with open(eval_folder+"/eval_input_output.pickle", "rb") as f:
    
    input_choices ,output_choices= pickle.load(f)
    
print(input_choices[:10])

print(output_choices[:10])

['お仕事 ふぁいと です !', '卵 は こないだ 渡し たろ う !', 'そろそろ ねむり ます おやすみなさい です', 'この カオナシ の クセ 強い w', 'アレェ !? やめ た の ? てか 元 から やっ て ない の ?', 'わかる', '黒 パンツ でし た ね', 'バイツ', 'を 作成 し た よ', 'マヨ が 壊れ た']
['あっ仕事は終わりましたw', 'あっそっかこれねすぐ食べちゃった', 'おやすみなさい。', 'これは?', 'やめたというか熱がねw', 'でもラクス派やねんw', '黒と言うか透明と言うかw', '新幹線できて', '貧乳に片思いって感じで草', '家の猫のまね']


### ファイル名注意

In [17]:
import pandas as pd
df = pd.DataFrame(columns=list(metrics_dict.keys()))
df.to_csv('mecab3min/to_csv_out5.csv')
print(df)

Empty DataFrame
Columns: [Bleu_1, Bleu_2, Bleu_3, Bleu_4, METEOR, ROUGE_L, CIDEr, SkipThoughtCS, EmbeddingAverageCosineSimilarity, EmbeddingAverageCosineSimilairty, VectorExtremaCosineSimilarity, GreedyMatchingScore]
Index: []


In [18]:
import os
import re
pathPos = './data/save/focal_loss_model/mecab_tagged.txt/4-4_256_0.2/'
fileList = os.listdir(pathPos)
print(len(fileList))
files_file = [f for f in fileList if os.path.isfile(os.path.join(pathPos,f))]
print(len(files_file))
print(files_file)

checkpoint_list = []

for f in files_file:
    numStr = re.search(r'\d+',f)
    numStr = int(numStr.group())
    checkpoint_list.append(numStr)
checkpoint_list.sort()
print()
print(len(checkpoint_list))
print(checkpoint_list)

64
63
['210000_checkpoint.tar', '320000_checkpoint.tar', '420000_checkpoint.tar', '435000_checkpoint.tar', '390000_checkpoint.tar', '72000_checkpoint.tar', '385000_checkpoint.tar', '260000_checkpoint.tar', '485000_checkpoint.tar', '180000_checkpoint.tar', '490000_checkpoint.tar', '128000_checkpoint.tar', '450000_checkpoint.tar', '445000_checkpoint.tar', '350000_checkpoint.tar', '30000_checkpoint.tar', '225000_checkpoint.tar', '230000_checkpoint.tar', '124000_checkpoint.tar', '300000_checkpoint.tar', '415000_checkpoint.tar', '400000_checkpoint.tar', '52000_checkpoint.tar', '240000_checkpoint.tar', '280000_checkpoint.tar', '160000_checkpoint.tar', '465000_checkpoint.tar', '470000_checkpoint.tar', '370000_checkpoint.tar', '4000_checkpoint.tar', '250000_checkpoint.tar', '68000_checkpoint.tar', '360000_checkpoint.tar', '375000_checkpoint.tar', '15000_checkpoint.tar', '290000_checkpoint.tar', '170000_checkpoint.tar', '475000_checkpoint.tar', '460000_checkpoint.tar', '220000_checkpoint.tar', 

In [ ]:
from nlgeval import compute_metrics
import pandas as pd

print(corpus_name)
# Configure models
model_name = "focal_loss_model"
attn_model = 'dot'
# attn_model = 'general'
# attn_model = "concat"
hidden_size = 256
encoder_n_layers = 4
decoder_n_layers = 4
dropout = 0.1
batch_size = 512
teacher_forcing_ratio=0.2

# Set checkpoint to load from; set to None if starting from scratch
loadFilename = None

#1000,2000,3000,4000,5000,6000,7000,8000,9000,15000,30000,52000,56000,60000,64000,68000,72000,96000,100000,102000,104000,120000,124000,128000,129000,148000,150000,151000,152000,153000,154000,155000,156000,157000,158000
#159000,160000,161000,162000,163000,164000,165000,166000,167000,168000,169000,170000,171000,172000,173000
#174000,175000,176000,177000,178000,179000,180000,181000,182000,183000,184000,185000,186000,187000,188000,189000,190000,191000,192000,193000,194000,195000,196000,197000
# checkpoint_list = [4000,15000,30000,52000,56000,68000,72000,100000,104000,120000,124000,128000,148000,150000,151000,152000,153000,155000,156000,157000,158000,
#                    159000,160000,161000,162000,163000,164000,165000,166000,167000,168000,169000,170000,171000,172000,173000,174000,175000,176000,178000,179000,
#                    180000,181000,182000,183000,184000,185000,186000,187000,188000,189000,190000,191000,192000,193000,194000,195000,196000,197000]

for checkpoint_iter in checkpoint_list:
    
    print('test',checkpoint_iter,'iter')

    loadFilename = os.path.join(save_dir, model_name, corpus_name,
                               '{}-{}_{}_{}'.format(encoder_n_layers, decoder_n_layers, hidden_size, teacher_forcing_ratio),
                               '{}_checkpoint.tar'.format(checkpoint_iter))


    # Load model if a loadFilename is provided
    if loadFilename:
        print("load save file")
        # If loading on same machine the model was trained on
        checkpoint = torch.load(loadFilename)
        # If loading a model trained on GPU to CPU
        # checkpoint = torch.load(loadFilename, map_location=torch.device('cpu'))
        encoder_sd = checkpoint["en"]
        decoder_sd = checkpoint["de"]
        encoder_optimizer_sd = checkpoint["en_opt"]
        decoder_optimizer_sd = checkpoint["de_opt"]
        embedding_sd = checkpoint["embedding"]
        voc.__dict__ = checkpoint["voc_dict"]


    print("Building encoder and decoder ...")
    # Initialize word embeddings
    embedding = nn.Embedding(voc.num_words, hidden_size)
    if loadFilename:
        embedding.load_state_dict(embedding_sd)
    # Initialize encoder & decoder models
    encoder = EncoderRNN(hidden_size, embedding, encoder_n_layers, dropout)
    decoder = AttnDecoderRNN(
        attn_model, embedding, hidden_size, voc.num_words, decoder_n_layers, dropout
    )
    if loadFilename:
        encoder.load_state_dict(encoder_sd)
        decoder.load_state_dict(decoder_sd)
    # Use appropriate device
    encoder = encoder.to(device)
    decoder = decoder.to(device)
    print("Models built and ready to go!")
    
    
    
    encoder.eval()
    decoder.eval()
    searcher = BeamSearchDecoder(encoder, decoder, 5, 5)
    hyp_sentences = []
    
    
    for input_sentence in input_choices:
        input_sentence = normalizeString(input_sentence)
        output_words = evaluate(encoder, decoder, searcher, voc, input_sentence)
        output_words[:] = [
            x for x in output_words if not (x == "EOS" or x == "PAD" or x=="SOS")
        ]
        hyp_sentences.append("".join(output_words))

        
    with open(eval_folder+'/hyp.txt','w') as f:
        f.write('\n'.join(hyp_sentences))
        
        
    eval_folder = 'mecab3min'
    metrics_dict = compute_metrics(hypothesis=eval_folder+'/hyp.txt',
                                   references=[eval_folder+'/ref.txt'])
    
    df = pd.read_csv('mecab3min/to_csv_out5.csv', index_col=0)
    nums = metrics_dict.values()
    df.loc[checkpoint_iter] = list(map(round,nums,[6]*len(nums)))
    df.to_csv('mecab3min/to_csv_out5.csv')


mecab_tagged.txt
test 4000 iter
load save file
Building encoder and decoder ...
Models built and ready to go!
Bleu_1: 0.000200
Bleu_2: 0.000014
Bleu_3: 0.000006
Bleu_4: 0.000004
METEOR: 0.017348
ROUGE_L: 0.000200
CIDEr: 0.000500
SkipThoughtsCosineSimilarity: 0.697018
EmbeddingAverageCosineSimilarity: 0.582509
EmbeddingAverageCosineSimilairty: 0.582509
VectorExtremaCosineSimilarity: 0.508448
GreedyMatchingScore: 0.850139
test 15000 iter
load save file
Building encoder and decoder ...
Models built and ready to go!
Bleu_1: 0.000000
Bleu_2: 0.000000
Bleu_3: 0.000000
Bleu_4: 0.000000
METEOR: 0.025073
ROUGE_L: 0.000000
CIDEr: 0.000000
SkipThoughtsCosineSimilarity: 0.673066
EmbeddingAverageCosineSimilarity: 0.529561
EmbeddingAverageCosineSimilairty: 0.529561
VectorExtremaCosineSimilarity: 0.443988
GreedyMatchingScore: 0.832434
test 30000 iter
load save file
Building encoder and decoder ...
Models built and ready to go!
Bleu_1: 0.000000
Bleu_2: 0.000000
Bleu_3: 0.000000
Bleu_4: 0.000000
METEOR

In [ ]:
Bleu_1: 0.003000
Bleu_2: 0.000055
Bleu_3: 0.000014
Bleu_4: 0.000007
METEOR: 0.036579
ROUGE_L: 0.003000
CIDEr: 0.007500
SkipThoughtsCosineSimilarity: 0.736541
EmbeddingAverageCosineSimilarity: 0.633790
EmbeddingAverageCosineSimilairty: 0.633790
VectorExtremaCosineSimilarity: 0.562120
GreedyMatchingScore: 0.868408